In [67]:
# Main notebook for playing around looking at different things in data
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab20"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab20.colors)
plt.rcParams['lines.markersize'] = 10
%matplotlib widget
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]


saveFigures is set to: True
Done loading packages


In [2]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath


for subdir, dirs, files in os.walk(rootdir):
    if not len(files) == 0:
        latestdir = subdir
        latestDate = pd.to_datetime(subdir[-10:])

print(latestdir)
print(latestDate)

c:\Users\rakrpe\Documents\PandemiX\GithubRepos\PandemiX\DanskeData/ssi_data\SSI_data_2021-04-26
2021-04-26 00:00:00


In [3]:
# Load the data
fulldfPCR = pd.read_csv(latestdir+'/Test_pos_over_time.csv',delimiter = ';',dtype=str)
fulldf = pd.read_csv(latestdir+'/Test_pos_over_time_antigen.csv',delimiter = ';',dtype=str)

# Cut out the last two rows of summary numbers
dfPCR = fulldfPCR.iloc[:-2,:].copy()
df = fulldf.iloc[:-2,:].copy()

In [4]:
dfPCR["PosPct"] = pd.to_numeric(dfPCR["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
dfPCR["Date"] = dfPCR["Date"].astype('datetime64[D]')
df["PosPct"] = pd.to_numeric(df["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
df["Date"] = df["Date"].astype('datetime64[D]')

rows_to_fix_period_in = ["NewPositive","NotPrevPos","PrevPos","Tested","Tested_kumulativ"]
for name in rows_to_fix_period_in:
    df[name] = pd.to_numeric(df[name].astype(str).apply(lambda x: x.replace('.','')))
    dfPCR[name] = pd.to_numeric(dfPCR[name].astype(str).apply(lambda x: x.replace('.','')))


In [5]:
# Load the data
df2 = pd.read_csv(latestdir+'/Antigentests_pr_dag.csv',delimiter = ';')

# # Remove the first two rows which are from april 2020
# df2 = df2.iloc[2:,:].copy()

In [6]:
df2["Dato"] = df2["Dato"].astype('datetime64[D]')
df2["Andel_AGpos_PCRkonf_pct"] = pd.to_numeric(df2["Andel_AGpos_PCRkonf_pct"].astype(str).apply(lambda x: x.replace(',','.')))
df2["Andel_AGminusPCRkonf_pct"] = pd.to_numeric(df2["Andel_AGminusPCRkonf_pct"].astype(str).apply(lambda x: x.replace(',','.')))

In [7]:
# print(df2.columns.values)
df2.tail()

,Dato,AG_testede,AG_pos,AGpos_PCRkonf,Andel_AGpos_PCRkonf_pct,AGpos_minusPCRkonf,Andel_AGminusPCRkonf_pct,AGpos_PCRpos,AGposPCRneg,AGnegPCRpos,AGnegPCRneg
76,2021-04-18,145624,265,247,93.207547,18,6.792453,155,89,152,7942
77,2021-04-19,278449,312,276,88.461538,36,11.538462,151,122,166,12092
78,2021-04-20,249677,292,262,89.726027,30,10.273973,147,110,144,11779
79,2021-04-21,265130,281,257,91.459075,24,8.540925,136,119,150,14042
80,2021-04-22,331566,261,239,91.570881,22,8.429119,114,123,125,16152


In [8]:
# plt.figure()
# # plt.plot(df2.Dato,df2.AG_testede)
# # plt.plot(df2.Dato,df2.Andel_AGpos_PCRkonf_pct)
# plt.plot(df2.Dato,df2.AG_pos)
# # plt.plot(df2.Dato,df2.AGpos_PCRkonf)
# plt.plot(df2.Dato,df2.AGpos_PCRkonf+df2.AGpos_minusPCRkonf)
# # plt.plot(df2.Dato,df2.AGpos_PCRpos)
# # plt.plot(df2.Dato,df2.AGposPCRneg)
# # plt.plot(df2.Dato,df2.AGpos_PCRpos+df2.AGposPCRneg)
# plt.show()

In [9]:
# fig,(ax1,ax2) = plt.subplots(2,1)
fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)

ax1.fill_between(df2.Dato,df2.AG_pos,color='blue',label='AG pos, PCR testet')
ax1.fill_between(df2.Dato,df2.AGpos_minusPCRkonf,color='grey',label='AG pos, afventer PCR')

ax2.fill_between(df2.Dato,100*np.ones(df2.Dato.shape),color='xkcd:mint',label='AG pos, PCR neg')
ax2.fill_between(df2.Dato,100*np.divide(df2.AGpos_PCRpos,df2.AGpos_PCRkonf),color='g',label='AG pos, PCR pos')

numAGnegKonf = df2.AGnegPCRpos + df2.AGnegPCRneg

ax3.fill_between(df2.Dato,100*np.ones(df2.Dato.shape),color='xkcd:mint',label='AG neg, PCR neg')
ax3.fill_between(df2.Dato,100*np.divide(df2.AGnegPCRpos,numAGnegKonf),color='g',label='AG neg, PCR pos')

ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax3.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=100)
ax3.set_ylim(bottom=0,top=3)
ax1.set_xlim([df2.Dato[0],df2.iloc[-1,0]])
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
df.tail()

,Date,NewPositive,NotPrevPos,PosPct,PrevPos,Tested,Tested_kumulativ
378,2021-04-21,262,264448,0.1,491,264939,6773545
379,2021-04-22,241,330801,0.1,585,331386,7104931
380,2021-04-23,215,291630,0.1,483,292113,7397044
381,2021-04-24,172,162045,0.1,319,162364,7559408
382,2021-04-25,142,167904,0.1,306,168210,7727618


In [11]:
dfPCR.tail()

,Date,NewPositive,NotPrevPos,PosPct,PrevPos,Tested,Tested_kumulativ
450,2021-04-21,697,180617,0.4,6769,187386,25573985
451,2021-04-22,639,215339,0.3,9101,224440,25798425
452,2021-04-23,623,173829,0.4,6443,180272,25978697
453,2021-04-24,565,134819,0.4,4454,139273,26117970
454,2021-04-25,263,51650,0.5,1778,53428,26171398


In [12]:
# fig,ax1 = plt.subplots(1,1)
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

ax1.plot(df.Date,df.Tested,label='Antal AG tests')
ax1.plot(dfPCR.Date,dfPCR.Tested,label='Antal PCR tests')

# ax2.plot(df.Date,df.PosPct,label='Positiv procent, AG')
# ax2.plot(dfPCR.Date,dfPCR.PosPct,label='Positive procent, PCR')
ax2.plot(df.Date,100*np.divide(df.NewPositive,df.Tested),label='Positiv procent, AG')
ax2.plot(dfPCR.Date,100*np.divide(dfPCR.NewPositive,dfPCR.Tested),label='Positiv procent, PCR')


ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=1)

ax1.set_xlim(left=np.datetime64('2021-02-01'),right=date.today())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d %b\n%Y'))

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Age stuff

In [20]:
dfAge = pd.read_csv(latestdir+'/Cases_by_age.csv',delimiter = ';',dtype=str)

In [14]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath


conf = []
test = []
posp = []

for subdir, dirs, files in os.walk(rootdir):
    if not len(files) == 0:
        latestdir = subdir
        latestDate = pd.to_datetime(subdir[-10:])
        curdf = pd.read_csv(latestdir+'/Cases_by_age.csv',delimiter = ';',dtype=str)
        conf.append(curdf.iloc[9,1])
        test.append(curdf.iloc[9,2])
        posp.append(curdf.iloc[9,3])
        print(curdf)

      30-39                    26.656       545.598              4,9
4         40-49                    28.775       581.930              4,9
5         50-59                    28.451       588.805              4,8
6         60-69                    15.572       432.082              3,6
7         70-79                     9.772       319.884              3,1
8         80-89                     5.252       124.239              4,2
9           90+                     1.662        26.051              6,4
10        I alt                   196.540     4.172.251              4,7
   Aldersgruppe Antal_bekræftede_COVID-19 Antal_testede Procent_positive
0           0-9                    12.790       361.587              3,5
1         10-19                    30.566       559.932              5,5
2         20-29                    37.256       633.607              5,9
3         30-39                    26.768       546.248              4,9
4         40-49                    28.879       582.807

In [25]:
# dfAge = pd.read_csv''/Cases_by_age.csv',delimiter = ';',dtype=str)
latestdir

dfAge = pd.read_csv('c:\\Users\\rakrpe\\Documents\\PandemiX\\GithubRepos\\PandemiX\\DanskeData/ssi_data\\SSI_data_2021-04-06\\Cases_by_age.csv',delimiter = ';',dtype=str)

In [27]:
dfAge

,Aldersgruppe,Antal_bekræftede_COVID-19,Antal_testede,Procent_positive
0,0-9,17.004,411.910,"4,1"
1,10-19,36.065,608.757,"5,9"
2,20-29,43.793,679.566,"6,4"
3,30-39,32.409,596.068,"5,4"
4,40-49,34.396,637.766,"5,4"
5,50-59,33.504,651.292,"5,1"
6,60-69,18.247,487.720,"3,7"
7,70-79,11.219,369.509,"3,0"
8,80-89,5.850,140.967,"4,1"
9,90+,1.830,29.111,"6,3"


In [80]:
# dfDeaths = pd.read_excel("C://Users//rakrpe//Documents//PandemiX//GithubRepos//PandemiX//DanskeData//DeathsAgeDK//Deaths.xlsx")

In [31]:
dfDeaths = pd.read_excel('DeathsAgeDK/Deaths.xlsx')

In [33]:
dfDeaths = dfDeaths.transpose()

In [41]:
dfDeaths.columns = dfDeaths.iloc[0]

In [48]:
dfDeaths = dfDeaths.drop(['Aldersgruppe']) 

In [54]:
# plt.figure()
# plt.plot(dfDeaths)


curDates =  pd.to_datetime(dfDeaths.index,format='%d_%m_%Y')

In [79]:
# plt.figure()
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)

for colNames in dfDeaths.columns[:-1]:
    # plt.plot(curDates,dfDeaths['0-9'])
    ax1.plot(curDates,dfDeaths[colNames],label=colNames)
    ax2.plot(curDates[0:-1],np.diff(dfDeaths[colNames]),label=colNames)
    ax3.plot(curDates[0:-1],np.diff(dfDeaths[colNames]),label=colNames)

ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.grid()
ax2.grid()
ax3.grid()

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)
ax3.set_ylim(bottom=0,top=10)

# ax1.set_xlim(left=curDates[0],right=date.today())
ax1.set_xlim(right=date.today())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d %b\n%Y'))

ax1.set_ylabel('Kumulative dødsfald')
ax2.set_ylabel('Dødsfald')
ax3.set_ylabel('Dødsfald')

plt.tight_layout()

if saveFigures:
    fig.savefig('figs/DeathsByAge')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

24